In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_columns', None)  

# Download and process crime files

These files come from the [City of Milwaukee's Open Data portal](https://data.milwaukee.gov/dataset?q=crime&sort=score+desc%2C+metadata_modified+desc). The "current" file is updated daily and contains individual incidents dating to the start of 2021. The historical file contains incidents from 2005-2020.

## Download and process historical file (disregard this section after first run)

In [2]:
# ####### ONLY RUN IF FIRST TIME downloading historical data #################
# historical_url = 'https://data.milwaukee.gov/dataset/5a537f5c-10d7-40a2-9b93-3527a4c89fbd/resource/395db729-a30a-4e53-ab66-faeb5e1899c8/download/wibrarchive.csv'
# hist_raw = pd.read_csv(historical_url, low_memory=False)

crime_data_dest = 'data/crime/'

In [3]:
# hist_raw.head(5)

In [4]:
# hist_raw.info()

ReportedDateTime needs to get converted to datetime format.

In [5]:
# hist_raw.ReportedDateTime = pd.to_datetime(hist_raw.ReportedDateTime)

In [6]:
### write locally so we don't have to download each time
# hist_raw.to_csv(crime_data_dest + 'historical.csv', index=None)

In [7]:
###### RUN THIS ONLY IF NOT FIRST TIME ############
### read local copy
hist_raw = pd.read_csv(crime_data_dest + 'historical.csv', low_memory=False)

## Download current file (updated daily)

In [12]:
current_url = 'https://data.milwaukee.gov/dataset/e5feaad3-ee73-418c-b65d-ef810c199390/resource/87843297-a6fa-46d4-ba5d-cb342fb2d3bb/download/wibr.csv'
current_raw = pd.read_csv(current_url, low_memory=False)

crime_data_dest = 'data/crime/'

### write locally so we don't have to download if needing offline access
current_raw.to_csv(crime_data_dest + 'current.csv', index=None)

In [8]:
current_raw = pd.read_csv(crime_data_dest + 'current.csv', low_memory=False)

In [10]:
current_raw.head(1)

,IncidentNum,ReportedDateTime,ReportedYear,ReportedMonth,Location,WeaponUsed,ALD,NSP,POLICE,TRACT,WARD,ZIP,RoughX,RoughY,Arson,AssaultOffense,Burglary,CriminalDamage,Homicide,LockedVehicle,Robbery,SexOffense,Theft,VehicleTheft
0,211900041,2021-07-09 06:30:00,2021,7,3635 N 8TH ST,ASPHYXIATION,6.0,4.0,5.0,4500.0,118.0,53206.0,2.555342e+06,401918.959099,0,1,0,0,0,0,0,0,0,0


In [6]:
current_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19495 entries, 0 to 19494
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   IncidentNum       19495 non-null  int64  
 1   ReportedDateTime  19495 non-null  object 
 2   ReportedYear      19495 non-null  int64  
 3   ReportedMonth     19495 non-null  int64  
 4   Location          19170 non-null  object 
 5   WeaponUsed        6621 non-null   object 
 6   ALD               19260 non-null  float64
 7   NSP               11555 non-null  float64
 8   POLICE            19297 non-null  float64
 9   TRACT             19254 non-null  float64
 10  WARD              19277 non-null  float64
 11  ZIP               19369 non-null  float64
 12  RoughX            19369 non-null  float64
 13  RoughY            19369 non-null  float64
 14  Arson             19495 non-null  int64  
 15  AssaultOffense    19495 non-null  int64  
 16  Burglary          19495 non-null  int64 

## Combine current with historical

In [7]:
crime = hist_raw.append(current_raw)

In [8]:
## convert to datetime format
crime.ReportedDateTime = pd.to_datetime(crime.ReportedDateTime)

In [13]:
### add total column
crime['Total'] = crime.iloc[:,-10:].sum(axis=1)

In [15]:
## add violent/property columns
crime['Violent'] = crime.AssaultOffense + crime.Homicide\
                            + crime.Robbery + crime.SexOffense

crime['Property'] = crime.Total - crime.Violent

crime.head(1)

,IncidentNum,ReportedDateTime,ReportedYear,ReportedMonth,Location,WeaponUsed,ALD,NSP,POLICE,TRACT,WARD,ZIP,RoughX,RoughY,Arson,AssaultOffense,Burglary,CriminalDamage,Homicide,LockedVehicle,Robbery,SexOffense,Theft,VehicleTheft,Total,Violent,Property
0,163020125,2016-10-28 19:55:00,2016,10,S 10TH ST / W GRANT ST,ROCK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1,0,0,0,0,0,0,2.0,0,2.0


In [16]:
# write to csv
crime.to_csv(crime_data_dest + 'crime.csv', index=None)